Global Runoff Data Center (GRDC) https://portal.grdc.bafg.de/

Download process:
1. Pick sub-region(s) or station(s) of interest.
2. Submit data request form. Select to include watershed boundaries.
3. Download zip file from request email response.

Download components:
- stationbasins.geojson: metadata and geoemetry of the watershed upstream of the gauge.
- subregions.geojson: global geodataframe of subregions
- stationID_Q_.txt: time series of Q (m3/s) for each station

In [38]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import codebase
from multiprocessing import Pool

In [2]:
datadir = '/global/scratch/users/ann_scheliga/aux_dam_datasets/GRDC_CRB/'
basin = 'niger'

In [3]:
subregions_meta = gpd.read_file(datadir + basin + '_subregions.geojson')
stations_meta = gpd.read_file(datadir + basin + '_stationbasins.geojson')

In [9]:
grdc_ids = stations_meta['grdc_no']

In [57]:
grdc_id_OI = grdc_ids[77]
print(grdc_id_OI)
test_gpd , test_data = codebase.load_data.load_GRDC_station_data_by_ID(grdc_id_OI, filepath=datadir,basin_str=basin)
test_data

1634400.0


,Q m3s
Date,
2000-01-01,42.7
2000-01-02,42.3
2000-01-03,42.0
2000-01-04,41.3
2000-01-05,39.8
...,...
2002-11-17,135.0
2002-11-18,131.0
2002-11-19,127.0
